## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-05-19-06-50 +0000,nyt,"2026 Winter Olympics Guide: How to Watch, Sche...",https://www.nytimes.com/2026/02/05/sports/wint...
1,2026-02-05-19-04-35 +0000,startribune,"Despite judge’s order, ICE sent an immigrant t...",https://www.startribune.com/judge-orders-detai...
2,2026-02-05-19-04-31 +0000,nyt,White House Offers Shifting Story on Gabbard’s...,https://www.nytimes.com/2026/02/05/us/politics...
3,2026-02-05-19-03-10 +0000,nyt,2 Dead After Police Helicopter Crashes in Arizona,https://www.nytimes.com/2026/02/05/us/helicopt...
4,2026-02-05-19-01-35 +0000,nyt,Trump Strips Job Protections From Thousands of...,https://www.nytimes.com/2026/02/05/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2468/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,35
241,epstein,24
54,new,20
98,ai,11
8,ice,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
244,2026-02-05-03-27-44 +0000,bbc,Trump says America should move on from Epstein...,https://www.bbc.com/news/articles/cy4gj71z0m0o...,72
291,2026-02-04-23-11-15 +0000,nypost,Trump gets help in his critical minerals race ...,https://nypost.com/2026/02/04/us-news/house-pa...,70
261,2026-02-05-00-56-40 +0000,latimes,L.A. Olympics head Casey Wasserman withstood a...,https://www.latimes.com/california/story/2026-...,67
63,2026-02-05-17-19-02 +0000,nypost,Trump tears into Don Lemon for free speech def...,https://nypost.com/2026/02/05/us-news/trump-te...,66
33,2026-02-05-18-13-13 +0000,cbc,Trump administration moves to end job protecti...,https://www.cbc.ca/news/world/trump-opb-kupor-...,65


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
244,72,2026-02-05-03-27-44 +0000,bbc,Trump says America should move on from Epstein...,https://www.bbc.com/news/articles/cy4gj71z0m0o...
28,61,2026-02-05-18-20-27 +0000,nypost,Mamdani crows about ‘model’ partnership with H...,https://nypost.com/2026/02/05/us-news/mamdani-...
192,56,2026-02-05-11-00-00 +0000,nypost,Pro-ICE billboards calling out ‘liberal hypocr...,https://nypost.com/2026/02/05/us-news/pro-ice-...
280,50,2026-02-04-23-48-00 +0000,wsj,Federal Reserve governor Lisa Cook sees a grea...,https://www.wsj.com/economy/central-banking/fe...
123,47,2026-02-05-14-47-58 +0000,cbc,"U.S., Russia agree to re-establish military-to...",https://www.cbc.ca/news/world/united-states-ru...
10,45,2026-02-05-18-53-46 +0000,nypost,Cops reveal shocking timeline of Savannah Guth...,https://nypost.com/2026/02/05/us-news/cops-rev...
68,42,2026-02-05-17-15-17 +0000,nypost,Stocks plunge on AI spending fears as tech rou...,https://nypost.com/2026/02/05/business/stocks-...
263,42,2026-02-05-00-53-35 +0000,nypost,Nike under fed scrutiny for DEI initiatives th...,https://nypost.com/2026/02/04/business/nike-un...
45,37,2026-02-05-17-50-42 +0000,nyt,LA Olympics Chief Keeps Low Profile in Milan G...,https://www.nytimes.com/2026/02/05/world/europ...
63,31,2026-02-05-17-19-02 +0000,nypost,Trump tears into Don Lemon for free speech def...,https://nypost.com/2026/02/05/us-news/trump-te...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
